In [1]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()

20-03-04:02:53:32 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-04:02:53:37 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp_4x053pc.yml
20-03-04:02:53:38 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [ ]:
# example from github
def feature_sum(xs):    
    return [str(sum(x)) for x in xs]
#a = [[1., 2., 3.]]
#b = feature_sum(a)
#print(b)#['6.0']

In [2]:
def bubble_sort(lists):
    res = list()
    for l in range(len(lists)):    
        alist = lists[l]
        for j in range(len(alist)-1,0,-1):
            for i in range(j):
                if alist[i] > alist[i+1]:
                    alist[i], alist[i+1] = alist[i+1], alist[i]
        res.append(str(alist))
    return res
# a = [[1,7,3],]
# b = bubble_sort(a)
# print(b) # ['[1, 3, 7]']

In [3]:
from clipper_admin.deployers import python as python_deployer

In [5]:
python_deployer.deploy_python_closure(clipper_conn, name="sum-model", version=1, input_type="int", func=bubble_sort)

20-03-04:02:53:58 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpvqgyyka9clipper
20-03-04:02:53:58 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-03-04:02:53:58 INFO     [python.py:192] Python closure saved
20-03-04:02:53:58 INFO     [python.py:206] Using Python 3.6 base image
20-03-04:02:53:58 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpvqgyyka9clipper
20-03-04:02:53:58 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-03-04:02:53:58 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-03-04:02:53:58 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpvqgyyka9clipper /model/
20-03-04:02:53:58 INFO     [clipper_admin.py:539] [default-cluster]  ---> 5c162455006e
20-03-04:02:53:58 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 5c162455006e
20-03-04:02:53:58 INFO   

In [6]:
clipper_conn.register_application(name="hello-world", input_type="int", default_output="-1.0", slo_micros=100000)

20-03-04:02:54:14 INFO     [clipper_admin.py:236] [default-cluster] Application hello-world was successfully registered


In [7]:
clipper_conn.link_model_to_app(app_name="hello-world", model_name="sum-model")

20-03-04:02:54:15 INFO     [clipper_admin.py:303] [default-cluster] Model sum-model is now linked to application hello-world


In [8]:
clipper_conn.get_all_apps()

['hello-world']

In [9]:
clipper_conn.get_all_models()

['sum-model:1']

In [11]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/hello-world/predict", headers=headers, data=json.dumps({"input": [7,2,3]})).json()

{'query_id': 1, 'output': '[2 3 7]', 'default': False}

In [ ]:
clipper_conn.delete_application(name="hello-world")

In [ ]:
clipper_conn.unlink_model_from_app(app_name="hello-world", model_name="sum-model")

In [ ]:
clipper_conn.stop_all()